Big-endian is an order in which the "big end" (most significant value in the sequence) is stored first (at the lowest storage address). 

Little-endian is an order in which the "little end" (least significant value in the sequence) is stored first. 

For example, in a big-endian computer, the two bytes required for the hexadecimal number 4F52 would be stored as 4F52 in storage (if 4F is stored at storage address 1000, for example, 52 will be at address 1001). In a little-endian system, it would be stored as 524F (52 at address 1000, 4F at 1001).

In [2]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/types.h>
#include "asm.h"

#define BUFSIZE 32
#define FLAGSIZE 64

void win() {
  char buf[FLAGSIZE];
  FILE *f = fopen("flag.txt","r");
  if (f == NULL) {
    printf("Flag File is Missing. Problem is Misconfigured, please contact an Admin if you are running this on the shell server.\n");
    exit(0);
  }

  fgets(buf,FLAGSIZE,f);
  printf(buf);
}

void vuln(){
  char buf[BUFSIZE];
  gets(buf);

  printf("Okay, time to return... Fingers Crossed... Jumping to 0x%x\n", get_return_address());
}

int main(int argc, char **argv){

  setvbuf(stdout, NULL, _IONBF, 0);
  
  gid_t gid = getegid();
  setresgid(gid, gid, gid);

  puts("Please enter your string: ");
  vuln();
  return 0;
}

SyntaxError: invalid syntax (<ipython-input-2-3f9b849e1e6b>, line 11)

setvbuf(stdout, NULL, _IONBF, 0);

This first function call will set stdout's buffering type to unbuffered.

This means that anything written to stdout will imedialty be written to stdout. 
This bypasses the usual line buffering that is the default behavior.

gid_t gid = getegid();
setresgid(gid, gid, gid);

getegid() gets the effective group id of the calling proccess.

setresgid() sets the real GID, effective GID, and saved set-group-ID 
of the calling process (andalways  modifies  the  filesystem  GID  
to be the same as the effective GID), with the same  restrictions 
for unprivileged processes.
